<a href="https://colab.research.google.com/github/opencae/hpc-training-gpu-2023/blob/main/appendix_A_build_rapidcfd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RapidCFDのビルド



## GNU Flexのインストール
RapidCFDのビルドに必要となります。


In [ ]:
!apt-get install flex

## インストール先のディレクトリ作成

In [ ]:
!mkdir /root/RapidCFD

## インストール先のディレクトリ移動

In [ ]:
%cd /root/RapidCFD

## RapidCFD-devのレポジトリのクローン

In [ ]:
!git clone https://github.com/Atizar/RapidCFD-dev.git

## パッチファイル入手

In [ ]:
!wget https://gitlab.com/OpenCAE/Supercomputer-OpenFOAM-Training/-/raw/master/build/colab/Nvcc/patches/RapidCFD-dev.diff

## パッチの適用

In [ ]:
!patch -d RapidCFD-dev -b -p1 < RapidCFD-dev.diff

## RapidCFD-devディレクトリ移動

In [ ]:
%cd RapidCFD-dev

## コンパイラオプションの変更

ハードウェア・アクセラレータがTesla T4(Turing)の場合には、NVIDIA architecture nameを`sm_75`とします。

* NVIDIA GPU の NVIDIA architectures のまとめ https://qiita.com/k_ikasumipowder/items/1142dadba01b42ac6012

In [ ]:
!for f in wmake/rules/linux64Nvcc/{c,c++};do sed -i 's/ -arch=sm_[0-9]*/ -arch=sm_75/' $f;done

## 環境設定およびビルド
割り当てられたCPUによって、 並列数`WM_NCOMPPROCS`の設定を変えください。
並列数が小さい場合には、非常に時間がかかります。


In [ ]:
!rm platforms/linux64NvccDPOpt/lib/libOpenFOAM.so;\
source etc/bashrc;\
export WM_NCOMPPROCS=2;\
LANG=C ./Allwmake 2>&1 | tee log.Allwmake

## 単精度版の環境設定およびビルド
もし単精度版をビルドする場合には、以下のようにします。

In [ ]:
!source /root/RapidCFD/RapidCFD-dev/etc/bashrc WM_PRECISION_OPTION=SP;\
export WM_NCOMPPROCS=2;\
LANG=C ./Allwmake 2>&1 | tee log.Allwmake.SP

## RapidCFD-devのアーカイブ


In [ ]:
%cd /root/RapidCFD
!tar Jcf RapidCFD-dev.tar.xz RapidCFD-dev

上記ファイルをファイル・ブラウからダウンロードするか、Googleドライブにコピーしておきます。